In [2]:
import pandas as pd
import numpy as np

prop_data = pd.read_csv("./prop_data_clean.csv", index_col=0)
print "imputing 1"
# temp_ind = prop_data.index[prop_data["heatingorsystemtypeid"] == 2.0]
# for i, row in prop_data.iterrows():
#     if i in temp_ind and np.isnan(row["airconditioningtypeid"]):
#         prop_data.loc[i, "airconditioningtypeid"] = 1.0

# t = prop_data.loc[prop_data["heatingorsystemtypeid"]==2 & prop_data["airconditioningtypeid"].isnull()]



prop_data["airconditioningtypeid"].fillna(-1, inplace=True)
print prop_data["airconditioningtypeid"].value_counts()

print prop_data[["airconditioningtypeid", "heatingorsystemtypeid"]].head()

print "imputed 1"

# In[32]:


duplicate_or_not_useful_cols = pd.Series(['calculatedbathnbr', 'assessmentyear', 'fullbathcnt', 
                    'regionidneighborhood', 'propertyzoningdesc', 'censustractandblock'])#,'finishedsquarefeet12'])
prop_data.drop(duplicate_or_not_useful_cols, axis=1, inplace=True)


# ## Imputing values

# In[33]:
print "imputing 2"

prop_data["buildingqualitytypeid"].fillna(prop_data["buildingqualitytypeid"].mean(), inplace=True)
prop_data["calculatedfinishedsquarefeet"].interpolate(inplace=True)
prop_data["heatingorsystemtypeid"].fillna(-1, inplace=True)
prop_data["lotsizesquarefeet"].fillna(prop_data["lotsizesquarefeet"].median(), inplace=True)
prop_data.drop(["numberofstories"], axis=1, inplace=True)
#removing propertycountylandusecode because it is not in interpretable format
prop_data.drop(["propertycountylandusecode"], axis=1, inplace=True)
prop_data["regionidcity"].interpolate(inplace=True)
prop_data["regionidzip"].interpolate(inplace=True)
prop_data["yearbuilt"].fillna(prop_data["yearbuilt"].mean(), inplace=True)

#impute structuretaxvaluedollarcnt, taxvaluedollarcnt, landtaxvaluedollarcnt, taxamount by interpolation
cols_to_interpolate = ["structuretaxvaluedollarcnt", "taxvaluedollarcnt", "landtaxvaluedollarcnt", "taxamount"]
for c in cols_to_interpolate:
    prop_data[c].interpolate(inplace=True)

    
print "imputed 2"

# prop_data.fillna(prop_data.mean(), inplace=True)


# In[34]:


#imputing garagecarcnt on basis of propertylandusetypeid
#All the residential places have 1 or 2 garagecarcnt, hence using random filling for those values.
print "imputing 3"
for i, row in prop_data.iterrows():
    if row["propertylandusetypeid"] in [261, 266] and np.isnan(row["garagecarcnt"]):
        prop_data.loc[i, "garagecarcnt"] = np.random.randint(1,3)

prop_data["garagecarcnt"].fillna(-1, inplace=True)

prop_data["garagecarcnt"].value_counts(dropna=False)

# In[35]:


#imputing garagetotalsqft using the garagecarcnt

for i, row in prop_data.iterrows():
    if np.isnan(row["garagetotalsqft"]) or row["garagetotalsqft"] == 0:
        if row["garagecarcnt"] == -1:
            prop_data.loc[i, "garagetotalsqft"] = -1
        elif row["garagecarcnt"] == 1:
            prop_data.loc[i, "garagetotalsqft"] = np.random.randint(180, 400)
        elif row["garagecarcnt"] == 2:
            prop_data.loc[i, "garagetotalsqft"] = np.random.randint(400, 720)
        elif row["garagecarcnt"] == 3:
            prop_data.loc[i, "garagetotalsqft"] = np.random.randint(720, 880)
        elif row["garagecarcnt"] == 4:
            prop_data.loc[i, "garagetotalsqft"] = np.random.randint(880, 1200)

prop_data["garagetotalsqft"].interpolate(inplace=True)

prop_data["garagetotalsqft"].value_counts(dropna=False)
print "imputed 3"


# In[36]:


cols_to_use = [col for col in prop_data_new.columns if col in ['calculatedfinishedsquarefeet', 'bedroomcnt', 'lotsizesquarefeet']]

# cols_to_use = corr_df_sel.col_labels.tolist()

temp_df = prop_data[cols_to_use]
corrmat = temp_df.corr(method="spearman")
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sb.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()


# In[37]:


for column in prop_data:
    print column, "\n",prop_data[column].value_counts(dropna=False), "\n\n\n********"


# In[38]:


#imputing unitcnt using propertylandusetypeid
print "imputing 4"
for i, row in prop_data.iterrows():
    if np.isnan(row["unitcnt"]):
        if row["propertylandusetypeid"] in [261, 266, 269]:
            prop_data.loc[i, "unitcnt"] = 1
        elif row["propertylandusetypeid"] == 246:
            prop_data.loc[i, "unitcnt"] = 2
        elif row["propertylandusetypeid"] == 247:
            prop_data.loc[i, "unitcnt"] = 3
        elif row["propertylandusetypeid"] == 248:
            prop_data.loc[i, "unitcnt"] = 4
            
prop_data["unitcnt"].fillna(1, inplace=True)

prop_data["unitcnt"].value_counts(dropna=False)

print "imputed 4"


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jay/.local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


imputing 1
-1.0     2173698
 1.0      742364
 13.0      58457
 5.0        8795
 11.0       1818
 12.0         59
 9.0          19
 3.0           7
Name: airconditioningtypeid, dtype: int64
   airconditioningtypeid  heatingorsystemtypeid
0                   -1.0                    NaN
1                   -1.0                    NaN
2                   -1.0                    NaN
3                   -1.0                    NaN
4                   -1.0                    NaN
imputed 1
imputing 2
imputed 2
imputing 3


KeyboardInterrupt: 